In [1]:
!pip3 install -U keras-tuner
import keras_tuner
from kerastuner.tuners import RandomSearch

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import keras as k
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import classification_report, confusion_matrix
import json

C:\Users\santo\AppData\Local\Temp/ipykernel_9172/338409175.py:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [2]:
dataset_url = 'https://www.openml.org/data/get_csv/4965303/flare.arff' 

dataset = np.genfromtxt(dataset_url, delimiter=',', skip_header=1)
x = dataset[:,:-1]
x_size = x.shape[1]
y = dataset[:,-1]

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x)
x = scaler.transform(x)

from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x, y, stratify=y, test_size=0.2)

from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = {0: class_weights[0], 1: class_weights[1]}
class_weights

{0: 0.6025459688826026, 1: 2.9379310344827587}

In [3]:
metrics = [keras.metrics.BinaryAccuracy(name='ACC')]
import tensorflow as tf
import keras.backend as K

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

In [4]:
def work(build_model, epochs=50):
  !rm -rf untitled_project/
  tuner = RandomSearch(build_model, objective='val_ACC', max_trials=50, executions_per_trial=3)
  tuner.search_space_summary()
  tuner.search(x_train, y_train, epochs=epochs, class_weight=class_weights, validation_data=(x_val, y_val), verbose=0)
  tuner.results_summary()
  best_nn = tuner.get_best_models(num_models=1)
  y_pred = best_nn[0].predict(x_val)
  y_pred = y_pred > 0.5
  print(confusion_matrix(y_val, y_pred))
  print(classification_report(y_val, y_pred, zero_division=0))
  config = json.dumps(best_nn[0].get_config(), indent=2)
  print(config)

### Loss = binary_crossentropy

# Santos

https://peltarion.com/knowledge-center/documentation/modeling-view/build-an-ai-model/loss-functions

Mira esto y explica binary_crossentropy. La ultima capa tiene que ser sigmoid.

Luego si tienes tiempo, hace lo mismo para categoricalcrossentropy, y la funcion de activacion para ultima capa tiene que ser softmax

In [5]:
def build_model_binary_1(hp):
    nn = Sequential()
    nn.add(Dense(units=hp.Int(f"units", 1, 100), activation=hp.Choice(f"activations", ["relu", "tanh", 'sigmoid'])))
    nn.add(Dense(1, activation="sigmoid"))
    nn.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])), loss=f1_loss, metrics=metrics)
    return nn

work(build_model_binary_1)

INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json


"rm" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json
Search space summary
Default search space size: 3
units (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 100, 'step': 1, 'sampling': None}
activations (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
INFO:tensorflow:Oracle triggered exit
Results summary
Results in .\untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
units: 24
activations: sigmoid
learning_rate: 0.001
Score: 0.8302180568377177
Trial summary
Hyperparameters:
units: 72
activations: sigmoid
learning_rate: 0.0001
Score: 0.827102780342102
Trial summary
Hyperparameters:
units: 60
activations: relu
learning_rate: 0.01
Score: 0.822429895401001
Trial summary
Hyperparameters:
units: 8
activations: tanh
learning_rate: 0.01
Score: 0.8099688490231832
Trial summary

In [6]:
def build_model_binary_2(hp):
    nn = Sequential()
    nn.add(Dense(units=hp.Int(f"units_1", 1, 100), activation=hp.Choice(f"activation_1", ["relu", "tanh", 'sigmoid'])))
    nn.add(Dense(units=hp.Int(f"units_2", 1, 100), activation=hp.Choice(f"activation_2", ["relu", "tanh", 'sigmoid'])))
    nn.add(Dense(1, activation="sigmoid"))
    nn.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])), loss=f1_loss, metrics=metrics)
    return nn

work(build_model_binary_2)

INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json


"rm" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json
Search space summary
Default search space size: 3
units (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 100, 'step': 1, 'sampling': None}
activations (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
INFO:tensorflow:Oracle triggered exit
Results summary
Results in .\untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
units: 24
activations: sigmoid
learning_rate: 0.001
Score: 0.8302180568377177
Trial summary
Hyperparameters:
units: 72
activations: sigmoid
learning_rate: 0.0001
Score: 0.827102780342102
Trial summary
Hyperparameters:
units: 60
activations: relu
learning_rate: 0.01
Score: 0.822429895401001
Trial summary
Hyperparameters:
units: 8
activations: tanh
learning_rate: 0.01
Score: 0.8099688490231832
Trial summary

In [7]:
def build_model_binary_3(hp):
    nn = Sequential()
    nn.add(Dense(units=hp.Int(f"units_1", 1, 100), activation=hp.Choice(f"activation_1", ["relu", "tanh", 'sigmoid'])))
    nn.add(Dense(units=hp.Int(f"units_2", 1, 100), activation=hp.Choice(f"activation_2", ["relu", "tanh", 'sigmoid'])))
    nn.add(Dense(units=hp.Int(f"units_3", 1, 100), activation=hp.Choice(f"activation_3", ["relu", "tanh", 'sigmoid'])))
    nn.add(Dense(1, activation="sigmoid"))
    nn.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])), loss=f1_loss, metrics=metrics)
    return nn

work(build_model_binary_3)

INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json


"rm" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json
Search space summary
Default search space size: 3
units (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 100, 'step': 1, 'sampling': None}
activations (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
INFO:tensorflow:Oracle triggered exit
Results summary
Results in .\untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
units: 24
activations: sigmoid
learning_rate: 0.001
Score: 0.8302180568377177
Trial summary
Hyperparameters:
units: 72
activations: sigmoid
learning_rate: 0.0001
Score: 0.827102780342102
Trial summary
Hyperparameters:
units: 60
activations: relu
learning_rate: 0.01
Score: 0.822429895401001
Trial summary
Hyperparameters:
units: 74
activations: sigmoid
learning_rate: 0.01
Score: 0.8099688490231832
Trial sum

In [8]:
def build_model_binary_4(hp):
    nn = Sequential()
    nn.add(Dense(units=hp.Int(f"units_1", 1, 100), activation=hp.Choice(f"activation_1", ["relu", "tanh", 'sigmoid'])))
    nn.add(Dense(units=hp.Int(f"units_2", 1, 100), activation=hp.Choice(f"activation_2", ["relu", "tanh", 'sigmoid'])))
    nn.add(Dense(units=hp.Int(f"units_3", 1, 100), activation=hp.Choice(f"activation_3", ["relu", "tanh", 'sigmoid'])))
    nn.add(Dense(units=hp.Int(f"units_4", 1, 100), activation=hp.Choice(f"activation_4", ["relu", "tanh", 'sigmoid'])))
    nn.add(Dense(1, activation="sigmoid"))
    nn.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])), loss=f1_loss, metrics=metrics)
    return nn

work(build_model_binary_4)

INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json


"rm" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json
Search space summary
Default search space size: 3
units (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 100, 'step': 1, 'sampling': None}
activations (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
INFO:tensorflow:Oracle triggered exit
Results summary
Results in .\untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
units: 24
activations: sigmoid
learning_rate: 0.001
Score: 0.8302180568377177
Trial summary
Hyperparameters:
units: 72
activations: sigmoid
learning_rate: 0.0001
Score: 0.827102780342102
Trial summary
Hyperparameters:
units: 60
activations: relu
learning_rate: 0.01
Score: 0.822429895401001
Trial summary
Hyperparameters:
units: 74
activations: sigmoid
learning_rate: 0.01
Score: 0.8099688490231832
Trial sum

In [9]:
def build_model_binary_5(hp):
    nn = Sequential()
    nn.add(Dense(units=hp.Int(f"units_1", 1, 100), activation=hp.Choice(f"activation_1", ["relu", "tanh", 'sigmoid'])))
    nn.add(Dense(units=hp.Int(f"units_2", 1, 100), activation=hp.Choice(f"activation_2", ["relu", "tanh", 'sigmoid'])))
    nn.add(Dense(units=hp.Int(f"units_3", 1, 100), activation=hp.Choice(f"activation_3", ["relu", "tanh", 'sigmoid'])))
    nn.add(Dense(units=hp.Int(f"units_4", 1, 100), activation=hp.Choice(f"activation_4", ["relu", "tanh", 'sigmoid'])))
    nn.add(Dense(units=hp.Int(f"units_5", 1, 100), activation=hp.Choice(f"activation_5", ["relu", "tanh", 'sigmoid'])))
    nn.add(Dense(1, activation="sigmoid"))
    nn.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])), loss=f1_loss, metrics=metrics)
    return nn

work(build_model_binary_5)

INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json


"rm" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json
Search space summary
Default search space size: 3
units (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 100, 'step': 1, 'sampling': None}
activations (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
INFO:tensorflow:Oracle triggered exit
Results summary
Results in .\untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
units: 24
activations: sigmoid
learning_rate: 0.001
Score: 0.8302180568377177
Trial summary
Hyperparameters:
units: 72
activations: sigmoid
learning_rate: 0.0001
Score: 0.827102780342102
Trial summary
Hyperparameters:
units: 60
activations: relu
learning_rate: 0.01
Score: 0.822429895401001
Trial summary
Hyperparameters:
units: 8
activations: tanh
learning_rate: 0.01
Score: 0.8099688490231832
Trial summary